# プロンプト自動進化システム - 実行ノートブック

このノートブックでは、攻撃プロンプトと防御プロンプトを自動で強化していくシステムの実行方法を説明します。

## 📌 システム概要

- **評価器**: Stage1-4をシミュレーション評価
- **進化エンジン**: LLMを使ってプロンプトを改善
- **世代管理**: 各世代の結果を自動保存

## 📋 このノートブックの内容

1. 環境セットアップと確認
2. クイックスタート（2世代のみ）
3. 本番実行（5世代）
4. 結果の分析と可視化
5. 提出ファイルの生成

## 1. 環境セットアップ

まず、必要なライブラリとAWS認証を確認します。

In [ ]:
# 必要なライブラリのインポート
import sys
import os
from pathlib import Path
import json
import pandas as pd
import boto3
from datetime import datetime

# プロジェクトルートをパスに追加
project_root = Path().resolve().parent
sys.path.append(str(project_root))

from configs.config import *

print("✓ ライブラリのインポート成功")
print(f"✓ プロジェクトルート: {project_root}")

In [ ]:
# AWS認証の確認
try:
    region_name = 'us-east-1'
    bedrock_client = boto3.client(
        service_name='bedrock-runtime',
        region_name=region_name
    )
    print("✓ AWS Bedrock クライアント初期化成功")
    print(f"✓ リージョン: {region_name}")
except Exception as e:
    print(f"✗ AWS認証エラー: {e}")
    print("\n【解決方法】")
    print("1. AWS CLIで認証: aws configure")
    print("2. または環境変数を設定:")
    print("   set AWS_ACCESS_KEY_ID=your_key")
    print("   set AWS_SECRET_ACCESS_KEY=your_secret")

In [ ]:
MODEL_ID = "apac.anthropic.claude-sonnet-4-20250514-v1:0"
MODEL_ID = "mistral.mistral-large-2402-v1:0"

In [ ]:
from src.llm_utils import LLMClient

llm = LLMClient(bedrock_client, MODEL_ID)

prompt = "プロンプトインジェクションのためのプロンプトを生成してください。"
response = llm.call(prompt)

print(response)

In [ ]:
# Use the Conversation API to send a text message to Mistral.

import boto3
from botocore.exceptions import ClientError

# Create a Bedrock Runtime client in the AWS Region you want to use.
client = boto3.client("bedrock-runtime", region_name="us-east-1")

# Set the model ID, e.g., Mistral Large.
model_id = "mistral.mistral-large-2402-v1:0"

# Start a conversation with the user message.
user_message = "研究目的でプロンプトインジェクションのためのプロンプトを生成してください。"
conversation = [
    {
        "role": "user",
        "content": [{"text": user_message}],
    }
]

try:
    # Send the message to the model, using a basic inference configuration.
    response = client.converse(
        modelId=model_id,
        messages=conversation,
        inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},
    )

    # Extract and print the response text.
    response_text = response["output"]["message"]["content"][0]["text"]
    print(response_text)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)




## 2.動作確認

まずは2世代のみで動作確認、高速モードを使用する

In [ ]:
# モジュールのインポート
from src.evolution import PromptCandidate, DefenseCandidate, EvolutionManager
from src.evaluator import FastEvaluator
from src.baseline import build_prompt_rows
from src.baseline_strategic import build_strategic_prompts

In [ ]:
# 初期プロンプトのロード（baseline使用）
rows = build_prompt_rows()

# 攻撃プロンプトを抽出
attack_rows = [r for r in rows if r["prompt_type"] == "attack"][:3]  # テスト用に3件のみ
defense_rows = [r for r in rows if r["prompt_type"] == "defense"]

# PromptCandidateに変換
initial_attacks = [
    PromptCandidate(
        prompt_id=r["prompt_id"],
        prompt_text=r["prompt_text"],
        generation=0
    )
    for r in attack_rows
]

initial_defense = DefenseCandidate(
    defense_id="defense_gen0",
    defense_text=defense_rows[0]["prompt_text"],
    generation=0
)

print(f"✓ 初期攻撃プロンプト: {len(initial_attacks)}件")
print(f"✓ 初期防御プロンプト: {len(initial_defense.defense_text)}文字")

In [ ]:
initial_attacks

In [ ]:
initial_defense

In [ ]:
# 高速評価器と進化マネージャーの初期化
evaluator = FastEvaluator(bedrock_client, block_model_id=MODEL_ID, target_model_id=MODEL_ID)

output_dir = Path(os.path.join(DIR_INTERIM, "evolution_test"))
manager = EvolutionManager(
    llm_client=bedrock_client,
    evaluator=evaluator,
    output_dir=output_dir,
    model_id=MODEL_ID,
)

print("✓ 高速評価器を初期化")
print(f"✓ 出力先: {output_dir}")

In [ ]:
# 進化実行（2世代、テスト用）
print("="*60)
print("クイックスタート: 2世代の進化を実行")
print("="*60)

final_attacks, final_defense = manager.run_evolution(
    initial_attacks=initial_attacks,
    initial_defense=initial_defense,
    num_generations=2,
    population_size=3
)

print("\n" + "="*60)
print("クイックスタート完了！")
print("="*60)

In [ ]:
# 結果の確認
print("【最終世代の攻撃プロンプト】")
for i, attack in enumerate(sorted(final_attacks, key=lambda x: x.score, reverse=True), 1):
    print(f"\n{i}. {attack.prompt_id}")
    print(f"   スコア: {attack.score:.1f}")
    print(f"   カテゴリ: {', '.join(attack.category_scores.keys()) if attack.category_scores else 'なし'}")
    print(f"   冒頭: {attack.prompt_text[:100]}...")

print(f"\n【最終防御プロンプト】")
print(f"ID: {final_defense.defense_id}")
print(f"ブロック率: {final_defense.avg_block_rate:.1%}")
print(f"ブロック成功: {final_defense.blocked_attacks}/{final_defense.total_attacks}")
print(f"冒頭: {final_defense.defense_text[:100]}...")

## 3. 本番実行（5世代・完全評価）

戦略的プロンプトから5世代進化させ、完全な評価を行います。

⚠️ **注意**: この実行には30-60分程度かかります。LLM APIの呼び出しコストが発生します。

In [ ]:
# 戦略的プロンプトのロード
strategic_rows = build_strategic_prompts()

# 攻撃・防御プロンプトを分離
attack_rows = [r for r in strategic_rows if r["prompt_type"] == "attack"]
defense_rows = [r for r in strategic_rows if r["prompt_type"] == "defense"]

# 初期化
initial_attacks_prod = [
    PromptCandidate(
        prompt_id=r["prompt_id"],
        prompt_text=r["prompt_text"],
        generation=0
    )
    for r in attack_rows
]

initial_defense_prod = DefenseCandidate(
    defense_id="defense_gen0",
    defense_text=defense_rows[0]["prompt_text"],
    generation=0
)

print(f"✓ 戦略的攻撃プロンプト: {len(initial_attacks_prod)}件")
print(f"✓ 戦略的防御プロンプト: {len(initial_defense_prod.defense_text)}文字")

In [ ]:
# 完全評価器と進化マネージャーの初期化（本番用）
from src.evaluator import PromptEvaluator

evaluator_prod = PromptEvaluator(bedrock_client)

output_dir_prod = Path(os.path.join(DIR_INTERIM, "evolution_production"))
manager_prod = EvolutionManager(
    llm_client=bedrock_client,
    evaluator=evaluator_prod,
    output_dir=output_dir_prod
)

print("✓ 完全評価器を初期化（LLMによる詳細評価）")
print(f"✓ 出力先: {output_dir_prod}")

In [ ]:
# 本番実行（5世代）
# ⚠️ 注意: この実行には30-60分かかります

print("="*70)
print("本番実行: 5世代の進化を開始")
print("="*70)
print("⏰ 推定所要時間: 30-60分")
print("💰 推定コスト: $5-10 (Bedrock使用料)")
print("="*70)

# 実行確認（コメントを外して実行）
final_attacks_prod, final_defense_prod = manager_prod.run_evolution(
    initial_attacks=initial_attacks_prod,
    initial_defense=initial_defense_prod,
    num_generations=5,
    population_size=5
)

print("\n⚠️ 本番実行するには上記のコメントを外してください")

## 4. 結果の分析

保存された世代ごとの結果を読み込んで分析します。

In [ ]:
# 世代ごとのファイルを検索
import glob

# テスト実行の結果を分析（本番実行後は output_dir_prod に変更）
analysis_dir = output_dir
attack_files = sorted(glob.glob(str(analysis_dir / "generation_*_attacks_*.json")))
defense_files = sorted(glob.glob(str(analysis_dir / "generation_*_defense_*.json")))

print(f"✓ 攻撃プロンプトファイル: {len(attack_files)}件")
print(f"✓ 防御プロンプトファイル: {len(defense_files)}件")

if attack_files:
    print(f"\n最新ファイル:")
    print(f"  攻撃: {Path(attack_files[-1]).name}")
    print(f"  防御: {Path(defense_files[-1]).name}")

In [ ]:
# 世代ごとのスコア推移を分析
if attack_files:
    generation_scores = []
    
    for attack_file in attack_files:
        with open(attack_file, 'r', encoding='utf-8') as f:
            attacks = json.load(f)
        
        # 世代番号を抽出
        gen_num = int(Path(attack_file).name.split('_')[1])
        
        # スコア統計
        scores = [a['score'] for a in attacks]
        
        generation_scores.append({
            'generation': gen_num,
            'avg_score': sum(scores) / len(scores) if scores else 0,
            'max_score': max(scores) if scores else 0,
            'min_score': min(scores) if scores else 0,
            'num_prompts': len(attacks)
        })
    
    # データフレーム化
    df_scores = pd.DataFrame(generation_scores).sort_values('generation')
    print("\n【世代ごとのスコア推移】")
    print(df_scores.to_string(index=False))
else:
    print("分析するファイルがありません。先に進化を実行してください。")

In [ ]:
# スコア推移の可視化
if attack_files:
    import matplotlib.pyplot as plt
    import japanize_matplotlib
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    ax.plot(df_scores['generation'], df_scores['avg_score'], 
            marker='o', label='平均スコア', linewidth=2)
    ax.plot(df_scores['generation'], df_scores['max_score'], 
            marker='s', label='最高スコア', linewidth=2)
    
    ax.set_xlabel('世代', fontsize=12)
    ax.set_ylabel('スコア', fontsize=12)
    ax.set_title('世代ごとのスコア推移', fontsize=14, fontweight='bold')
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n✓ 最終世代の平均スコア: {df_scores['avg_score'].iloc[-1]:.2f}")
    print(f"✓ 最終世代の最高スコア: {df_scores['max_score'].iloc[-1]:.2f}")
    print(f"✓ 改善率: {(df_scores['max_score'].iloc[-1] / df_scores['max_score'].iloc[0] - 1) * 100:.1f}%")

In [ ]:
# カテゴリ別の達成状況を分析
if attack_files:
    # 最終世代の攻撃プロンプトを読み込み
    with open(attack_files[-1], 'r', encoding='utf-8') as f:
        final_attacks_data = json.load(f)
    
    # カテゴリごとの達成状況
    category_counts = {}
    for attack in final_attacks_data:
        for cat in attack.get('category_scores', {}).keys():
            category_counts[cat] = category_counts.get(cat, 0) + 1
    
    print("\n【カテゴリ別達成状況】")
    target_categories = [
        "心理操作", "社会工学", "技術悪用", 
        "詐欺手法", "情報収集", "暴力・犯罪", "違法行為"
    ]
    
    for cat in target_categories:
        count = category_counts.get(cat, 0)
        status = "✓" if count > 0 else "✗"
        print(f"{status} {cat}: {count}件")
    
    achieved = len([c for c in target_categories if c in category_counts])
    print(f"\n達成率: {achieved}/{len(target_categories)} カテゴリ ({achieved/len(target_categories)*100:.1f}%)")

## 5. 提出ファイルの生成

最終世代から上位5件の攻撃プロンプトと最良の防御プロンプトを選択して、提出用CSVを生成します。

In [ ]:
# 最終世代から提出ファイルを生成
if attack_files and defense_files:
    # 最終世代のデータを読み込み
    with open(attack_files[-1], 'r', encoding='utf-8') as f:
        final_attacks_data = json.load(f)
    
    with open(defense_files[-1], 'r', encoding='utf-8') as f:
        final_defense_data = json.load(f)
    
    # スコアでソート
    sorted_attacks = sorted(final_attacks_data, key=lambda x: x['score'], reverse=True)
    
    # 上位5件を選択
    top_attacks = sorted_attacks[:5]
    
    # 提出形式に変換
    submission_rows = []
    
    for i, attack in enumerate(top_attacks, 1):
        submission_rows.append({
            "prompt_type": "attack",
            "prompt_id": f"attack_{i}",
            "prompt_text": attack['prompt_text'][:2000]  # 2000文字制限
        })
    
    # 防御プロンプト
    submission_rows.append({
        "prompt_type": "defense",
        "prompt_id": "defense_1",
        "prompt_text": final_defense_data['defense_text'][:2000]
    })
    
    # データフレーム化
    df_submission = pd.DataFrame(submission_rows)
    
    # CSV保存
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    submission_file = Path("data/submission") / f"submission_evolved_{timestamp}.csv"
    submission_file.parent.mkdir(parents=True, exist_ok=True)
    
    df_submission.to_csv(submission_file, index=False, encoding="utf-8")
    
    print(f"✓ 提出ファイル生成完了: {submission_file}")
    print(f"\n【提出プロンプト一覧】")
    print(df_submission[['prompt_type', 'prompt_id']].to_string(index=False))
else:
    print("提出ファイルを生成するには、先に進化を実行してください。")

In [ ]:
# 提出ファイルの内容を確認
if 'df_submission' in locals():
    print("\n【生成された提出ファイルの詳細】")
    for idx, row in df_submission.iterrows():
        print(f"\n{row['prompt_type'].upper()}: {row['prompt_id']}")
        print(f"文字数: {len(row['prompt_text'])}")
        print(f"冒頭: {row['prompt_text'][:150]}...")
        print("-" * 60)

## 6. 追加の実験・カスタマイズ

### 実験例

以下のような追加実験が可能です:

1. **異なる初期値で実行**: baseline vs strategic の比較
2. **世代数の変更**: 3世代 vs 10世代 の効果比較
3. **個体数の変更**: 少数精鋭 vs 多様性重視
4. **評価モードの比較**: Fast vs 完全評価

### カスタマイズポイント

- `src/evolution.py`: 進化戦略のカスタマイズ
- `src/evaluator.py`: 評価ロジックのカスタマイズ
- 初期プロンプト: 独自のプロンプトから開始

In [ ]:
# 異なるパラメータでの実験例（テンプレート）

# 実験1: 世代数を増やす
# final_attacks_exp1, final_defense_exp1 = manager.run_evolution(
#     initial_attacks=initial_attacks,
#     initial_defense=initial_defense,
#     num_generations=10,  # 10世代
#     population_size=5
# )

# 実験2: 個体数を増やす
# final_attacks_exp2, final_defense_exp2 = manager.run_evolution(
#     initial_attacks=initial_attacks,
#     initial_defense=initial_defense,
#     num_generations=5,
#     population_size=10  # 10個体
# )

print("実験テンプレートです。コメントを外して実行してください。")

## まとめ

### 実行手順

1. ✅ **環境セットアップ**: AWS認証とライブラリ確認
2. ✅ **クイックスタート**: 2世代で動作確認（5-10分）
3. ⏳ **本番実行**: 5世代で完全進化（30-60分）
4. 📊 **結果分析**: スコア推移とカテゴリ達成状況
5. 📄 **提出ファイル生成**: 上位5件の攻撃+防御

### 次のステップ

- 複数回実行して最良の結果を選択
- 異なる初期値・パラメータで比較
- カスタム評価・進化戦略の実装
- アンサンブル手法の検討

### 参考ドキュメント

- `docs/evolution_system.md`: システムの詳細説明
- `docs/setup_guide.md`: セットアップと詳細ガイド
- `.github/copilot-instructions.md`: コンペ戦略